In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/BirdData/BirdClassification.zip"

Archive:  /content/drive/MyDrive/BirdData/BirdClassification.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  


In [ ]:

train_path = "/content/train_data/train_data"
test_path = "/content/test_data/test_data"


In [ ]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')


Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input


In [ ]:
# Adding the preprocessing layer to the front of vgg

vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Train model with existing weights

for layer in vgg.layers:
  print(layer)

In [ ]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [ ]:
x = Flatten()(vgg.output)


In [ ]:
# output layer

prediction = Dense(16,activation='softmax')(x)

In [ ]:
# Create Vgg16 model

model = Model(inputs=vgg.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train, validation_data=test, epochs=4,
          steps_per_epoch=len(train), validation_steps=len(test))


Epoch 1/4
7/7 [==============================] - 349s 55s/step - loss: 3.4705 - accuracy: 0.2400 - val_loss: 3.0138 - val_accuracy: 0.2038
Epoch 2/4
7/7 [==============================] - 337s 52s/step - loss: 1.9782 - accuracy: 0.3467 - val_loss: 3.3675 - val_accuracy: 0.2166
Epoch 3/4
7/7 [==============================] - 334s 51s/step - loss: 0.9713 - accuracy: 0.7267 - val_loss: 3.0701 - val_accuracy: 0.3057
Epoch 4/4
7/7 [==============================] - 336s 52s/step - loss: 0.6449 - accuracy: 0.8200 - val_loss: 3.0168 - val_accuracy: 0.3312


**ResNet50**

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
resnet = ResNet50(include_top=False,input_shape=(224,224,3))


94765736/94765736 [==============================] - 0s 0us/step


In [ ]:

for layer in resnet.layers:
  layer.trainable=False

In [ ]:

for layer in resnet.layers:
  print(layer)

In [ ]:

x = Flatten()(resnet.output)

In [ ]:
out = Dense(16, activation='softmax')(x)

In [ ]:

res_model = Model(inputs=resnet.input,outputs=out)

In [ ]:

res_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
res_model.fit(train,epochs=5,validation_data=test,steps_per_epoch=len(train),
              validation_steps=len(test))

Epoch 1/5
7/7 [==============================] - 223s 35s/step - loss: 3.2044 - accuracy: 0.2733 - val_loss: 4.3514 - val_accuracy: 0.1274
Epoch 2/5
7/7 [==============================] - 215s 34s/step - loss: 2.4536 - accuracy: 0.3133 - val_loss: 3.7397 - val_accuracy: 0.1847
Epoch 3/5
7/7 [==============================] - 216s 34s/step - loss: 2.4149 - accuracy: 0.2533 - val_loss: 3.5883 - val_accuracy: 0.1592
Epoch 4/5
7/7 [==============================] - 160s 25s/step - loss: 2.1241 - accuracy: 0.3733 - val_loss: 3.7156 - val_accuracy: 0.2484
Epoch 5/5
7/7 [==============================] - 213s 33s/step - loss: 2.0756 - accuracy: 0.3667 - val_loss: 3.7946 - val_accuracy: 0.1847
